In [9]:
import lightgbm as lgb

In [2]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
from dotenv import load_dotenv

from src.metrics import pearson_metric
from src.feature_engineering import compute_kmeans_clusters

load_dotenv()

True

In [3]:
%%time
dataset_dir = Path(os.environ['dataset_dir'])
full_data = pd.read_csv(dataset_dir / 'train.csv')
sample_data = pd.read_csv(dataset_dir / 'train_sample.csv', index_col=0)

CPU times: user 2min 30s, sys: 9.12 s, total: 2min 40s
Wall time: 2min 53s


In [22]:
data = full_data

In [23]:
data = data.set_index('row_id')

In [24]:
# train on earlier data, test on later data
train = data[data.time_id < 1000]
val = data.query("1000 <= time_id < 1100")
test = data.query("1000 <= time_id")

x_train, y_train = train.drop('target', axis=1), train[['time_id', 'target']]
x_val, y_val = val.drop('target', axis=1), val[['time_id', 'target']]
x_test, y_test = test.drop('target', axis=1), test[['time_id', 'target']]

In [43]:
train_data = lgb.Dataset(x_train.drop('time_id', axis=1), label=y_train.target, categorical_feature=['investment_id'])
test_data = lgb.Dataset(x_test.drop('time_id', axis=1), label=y_train.target, categorical_feature=['investment_id'])

In [44]:
param = {'objective': 'regression'}
bst = lgb.train(param, train_data, 100)

/miniconda/lib/python3.8/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Info] Total Bins 79991
[LightGBM] [Info] Number of data: 2433272, number of used features: 301
[LightGBM] [Info] Start training from score -0.024092


In [45]:
y_pred = bst.predict(x_test.drop('time_id', axis=1))

In [46]:
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
r2_score(y_test.target, y_pred)

0.009869813758294987

In [50]:
np.sqrt(np.mean((y_test.target - y_pred) ** 2))

0.9001908210276011